<a href="https://colab.research.google.com/github/vvrgit/AI/blob/main/Load_Forecasting_2021_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import required libraries

In [1]:
import numpy as np
import math as mt
import random
import pandas as pd
import matplotlib.pyplot as plt

## Import Data

In [4]:
data = pd.read_excel('/content/Active Power Load - 33_11KV - Godishala Substation  (1).xlsx')
data.head()

,DATE,TIME,VOLTAGE,CURRENT,PF,POWER (KW),"""WEEKEND/WEEKDAY""",SEASON,Temp (F),Humidity (%),Substation Shutdown,F1,F2,F3,F4,Jul-Oct-,Rainy,0
0,2021-01-01,01-00,11.6,102,0.96,1967.388015,0,1,65,90,NaN,NaN,NaN,NaN,NaN,Nov-Feb,Winter,1.0
1,NaT,02-00,11.6,102,0.96,1967.388015,0,1,65,90,NaN,NaN,NaN,NaN,NaN,Mar-Jun,Summer,2.0
2,NaT,03-00,11.6,102,0.96,1967.388015,0,1,65,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,04-00,11.3,130,0.96,2442.607331,0,1,78,49,NaN,NaN,NaN,NaN,NaN,Weekend,1,NaN
4,NaT,05-00,11.2,148,0.96,2756.205522,0,1,78,49,NaN,NaN,NaN,NaN,NaN,Weekday,0,NaN


In [ ]:
data.describe()

,VOLTAGE,CURRENT,PF,POWER (KW),"""WEEKEND/WEEKDAY""",SEASON,Temp (F),Humidity (%),Substation Shutdown,F1,F2,F3,F4,0
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.00000,8760.000000,8760.000000,66.0,25.0,71.0,97.0,46.0,2.000000
mean,11.550228,114.802854,0.929584,2129.985907,0.142466,0.99726,81.184132,67.726712,1.0,1.0,1.0,1.0,1.0,1.500000
std,0.179907,69.966107,0.058151,1302.278205,0.349547,0.81933,8.881827,20.684501,0.0,0.0,0.0,0.0,0.0,0.707107
min,11.000000,21.000000,0.690000,412.034103,0.000000,0.00000,50.000000,14.000000,1.0,1.0,1.0,1.0,1.0,1.000000
25%,11.400000,56.750000,0.900000,1062.656472,0.000000,0.00000,76.000000,52.000000,1.0,1.0,1.0,1.0,1.0,1.250000
50%,11.600000,92.000000,0.960000,1683.688485,0.000000,1.00000,81.000000,72.000000,1.0,1.0,1.0,1.0,1.0,1.500000
75%,11.700000,164.000000,0.970000,3025.768919,0.000000,2.00000,86.000000,87.000000,1.0,1.0,1.0,1.0,1.0,1.750000
max,12.000000,379.000000,1.000000,6306.206465,1.000000,2.00000,108.000000,102.000000,1.0,1.0,1.0,1.0,1.0,2.000000


In [5]:
load=data.iloc[:,5]
temp=data.iloc[:,8]
hum=data.iloc[:,9]
day_status=data.iloc[:,6]
season_status=data.iloc[:,7]

In [9]:
k=48
a=np.zeros((len(load)-48,9))
a.shape
a[:,4]=day_status.iloc[48:]
a[:,5]=season_status.iloc[48:]
a[:,6]=temp.iloc[48:]
a[:,7]=hum.iloc[48:]
a[:,8]=load.iloc[48:]
for i in range(8712):
  a[i,0],a[i,1],a[i,2],a[i,3]=load.iloc[k-1],load.iloc[k-2],load.iloc[k-24],load.iloc[k-48]
  k=k+1
a


array([[2175.94078853,  446.57465972, 1828.91574898, ...,   65.        ,
          92.        , 2236.75655649],
       [2236.75655649, 2175.94078853, 1828.91574898, ...,   65.        ,
          92.        , 2236.75655649],
       [2236.75655649, 2236.75655649, 1828.91574898, ...,   65.        ,
          92.        , 2354.48058578],
       ...,
       [1173.20461451, 1221.58079356,  898.72652303, ...,   65.        ,
          94.        , 1078.59133914],
       [1078.59133914, 1173.20461451, 2023.11155348, ...,   65.        ,
          94.        ,  908.28744349],
       [ 908.28744349, 1078.59133914, 2023.11155348, ...,   65.        ,
          94.        ,  839.83679557]])

In [10]:
df = pd.DataFrame(a, columns = ['T-1','T-2','T-24','T-48','DAY','SEASON','TEMP','HUMIDITY','T'])
df

,T-1,T-2,T-24,T-48,DAY,SEASON,TEMP,HUMIDITY,T
0,2175.940789,446.574660,1828.915749,1967.388015,1.0,1.0,65.0,92.0,2236.756556
1,2236.756556,2175.940789,1828.915749,1967.388015,1.0,1.0,65.0,92.0,2236.756556
2,2236.756556,2236.756556,1828.915749,1967.388015,1.0,1.0,65.0,92.0,2354.480586
3,2354.480586,2236.756556,1892.265507,2442.607331,1.0,1.0,77.0,52.0,2511.445958
4,2511.445958,2354.480586,2532.344883,2756.205522,1.0,1.0,77.0,52.0,2805.756031
...,...,...,...,...,...,...,...,...,...
8707,1240.667993,1011.621595,1160.855092,1154.281959,0.0,1.0,67.0,92.0,1221.580794
8708,1221.580794,1240.667993,1011.067338,965.055409,0.0,1.0,67.0,92.0,1173.204615
8709,1173.204615,1221.580794,898.726523,851.519478,0.0,1.0,65.0,94.0,1078.591339
8710,1078.591339,1173.204615,2023.111553,2668.094365,0.0,1.0,65.0,94.0,908.287443


## Missing Values Treatment

In [11]:
print(df.isnull().sum())

T-1         0
T-2         0
T-24        0
T-48        0
DAY         0
SEASON      0
TEMP        0
HUMIDITY    0
T           0
dtype: int64


## Data Split

In [12]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, train_size = 0.7, test_size = 0.3, random_state = 100)

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train= scaler.fit_transform(df_train)
df_test= scaler.fit_transform(df_test)

df_train

array([[0.06561008, 0.06892172, 0.12023797, ..., 0.60344828, 0.72727273,
        0.05367438],
       [0.31100572, 0.22348079, 0.30713329, ..., 0.63793103, 0.40909091,
        0.34791315],
       [0.14614022, 0.19605363, 0.15337913, ..., 0.63793103, 0.29545455,
        0.12792621],
       ...,
       [0.06919769, 0.08216827, 0.06948309, ..., 0.29310345, 0.88636364,
        0.06919769],
       [0.08520394, 0.09472491, 0.1025641 , ..., 0.55172414, 0.86363636,
        0.199209  ],
       [0.07019808, 0.07313026, 0.1503694 , ..., 0.53448276, 0.79545455,
        0.05142475]])

In [15]:
# Dividing the training data set into X and Y
X_train = df_train[:,0:8]
y_train = df_train[:,8]
X_test = df_test[:,0:8]
y_test = df_test[:,8]

In [25]:
y_test

array([0.27416627, 0.3335796 , 0.81357471, ..., 0.51767503, 0.17775577,
       0.26842455])

In [26]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [27]:
m = regressor.coef_
c = regressor.intercept_,
print("Estimated model weights, m:" , m)
print("Estimated model intercept, c:" , c)

Estimated model weights, m: [ 5.80348841e-01 -9.83822028e-02  3.21305035e-01  1.50728583e-01
 -1.89472969e-04 -1.61686237e-02 -1.18353625e-02 -6.86897489e-02]
Estimated model intercept, c: (0.06964815585650846,)


In [29]:
y_pred_test =  regressor.predict(X_test)
y_pred_train =  regressor.predict(X_train)
y_pred_test

array([0.29047772, 0.39060215, 0.84784879, ..., 0.44990721, 0.18209023,
       0.26728437])

In [30]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.07102471390179683
Mean square error 0.005044509984832093
Mean absolute error 0.042316549656241494


In [31]:
#Testing Accuracies
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.07163557863046116
Mean square error 0.005131656125720983
Mean absolute error 0.04450666015035177


In [32]:
import pickle
# Save the model
filename = 'linear_regression.pkl'
pickle.dump(regressor, open(filename, 'wb'))

In [34]:
list_of_columns = ['T-1','T-2','T-24','T-48','DAY','SEASON','TEMP','HUMIDITY']
input_data=pd.DataFrame(columns=list_of_columns)

input_data.at[0, 'T-1'] = int(input('ENTER PREVIOUS HOUR LOAD'))
input_data.at[0, 'T-2'] = int(input('ENTER TWO HOURS BEFORE LOAD'))
input_data.at[0, 'T-24'] = int(input('ENTER ONE DAY BEFORE LOAD'))
input_data.at[0, 'T-48'] = int(input('ENTER TWO DAY BEFORE LOAD'))
input_data.at[0, 'DAY'] = int(input('0: WEEKDAY AND 1:WEEKEND'))
input_data.at[0, 'SEASON'] = int(input('0:RAINY,1:WINTER,2:SUMMER'))
input_data.at[0, 'TEMP'] = int(input('ENTER TEMPARATURE'))
input_data.at[0, 'HUMIDITY'] = int(input('ENTER HUMIDITY'))



model = pickle.load(open('linear_regression.pkl', 'rb'))
prediction = model.predict(input_data)
result = prediction[0] #*(6306-432)+432
print('Predicted load in kW',result)

ENTER PREVIOUS HOUR LOAD2176
ENTER TWO HOURS BEFORE LOAD447
ENTER ONE DAY BEFORE LOAD1829
ENTER TWO DAY BEFORE LOAD1967
0: WEEKDAY AND 1:WEEKEND1
0:RAINY,1:WINTER,2:SUMMER1
ENTER TEMPARATURE65
ENTER HUMIDITY92
Tredicted load in kW 2095.9767990087553


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


# Decision Tree Regressor

In [48]:
# Fitting Decision Tree Regression to the dataset
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=None)
regressor.fit(X_train, y_train)
regressor.get_depth()

34

In [43]:
y_pred_test =  regressor.predict(X_test)
y_pred_train =  regressor.predict(X_train)

In [44]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.06563006004804028
Mean square error 0.004307304781909374
Mean absolute error 0.039150829888698495


In [45]:
#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.07232056566156829
Mean square error 0.005230264217609211
Mean absolute error 0.043845684749535985


In [46]:
import pickle
# Save the model
filename = 'decision_Tree.pkl'
pickle.dump(regressor, open(filename, 'wb'))

In [47]:
list_of_columns = ['T-1','T-2','T-24','T-48','DAY','SEASON','TEMP','HUMIDITY']
input_data=pd.DataFrame(columns=list_of_columns)

input_data.at[0, 'T-1'] = int(input('ENTER PREVIOUS HOUR LOAD'))
input_data.at[0, 'T-2'] = int(input('ENTER TWO HOURS BEFORE LOAD'))
input_data.at[0, 'T-24'] = int(input('ENTER ONE DAY BEFORE LOAD'))
input_data.at[0, 'T-48'] = int(input('ENTER TWO DAY BEFORE LOAD'))
input_data.at[0, 'DAY'] = int(input('0: WEEKDAY AND 1:WEEKEND'))
input_data.at[0, 'SEASON'] = int(input('0:RAINY,1:WINTER,2:SUMMER'))
input_data.at[0, 'TEMP'] = int(input('ENTER TEMPARATURE'))
input_data.at[0, 'HUMIDITY'] = int(input('ENTER HUMIDITY'))



model = pickle.load(open('decision_Tree.pkl', 'rb'))
prediction = model.predict(input_data)
result = prediction[0]*(6306-432)+432.
print('Tredicted load in kW',result)

ENTER PREVIOUS HOUR LOAD2176
ENTER TWO HOURS BEFORE LOAD447
ENTER ONE DAY BEFORE LOAD1829
ENTER TWO DAY BEFORE LOAD1967
0: WEEKDAY AND 1:WEEKEND1
0:RAINY,1:WINTER,2:SUMMER1
ENTER TEMPARATURE65
ENTER HUMIDITY92
Tredicted load in kW 5722.885492274611


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


# Thank You